RQ1: What are the frames used by CNN and FOX in the stories using candidates’ tweets as news sources during the 2016 and 2020 US presidential campaign?
-	Data analysis of frames in news stories (percent, other statistics) 
-	Comparison between the 2 networks, and also comparison between 2016 and 2020 (CNN2016-2020, and Fox2016-2020)

Create a table like this
             		CNN2016 	FOX2016 	CNN2020 	FOX2020
Frame 1			%	%		%		%
Frame 2
Frame 3
And so forth.....
Positive	-- include these 2, they are emotional frames
Negative 

Report differences, if any, CNN-FOX2016 and CNN-Fox2020, but also CNN2016-CNN2020, and FOX2016-FOX2020.


In [29]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [30]:
#open file
df=pd.read_csv('all.csv')
#cases STORYNUMBER 191 and 195 are removed since twitter had missingO values

In [31]:
#fixing coding errors
df['STORYEMOTIONALFRAMES'] = df['STORYEMOTIONALFRAMES'].replace(["3"], 'neutral')
df['TWEETEMOTIONALFRAMES'] = df['TWEETEMOTIONALFRAMES'].replace(["3"], 'neutral')
#unit of analysis is tweet
dftweet=df[['STORYNUMBER', 'CODER', 'YEAR', 'PUBLICATIONDATE', 'URL', 'PUBLISHER', "PUBLISHERYEAR",
       'NUMBEROFTWEETSBYCANDIDATES', 'NUMBEROFTWEETSFROMOTHERS', 'TWEETBY',
       'TWEETPRESENTATION', 'TWEETGENERICFRAMESDOMINANT','TWEETEMOTIONALFRAMES',
       'TWEETCONTAINSFALSESTATEMENT', 'TWEETCLAIMSSOMETHINGELSEISFALSE',
       'TWEETTOPIC', 'TWEETROLETOTHESTORY', 'STORYROLETOTWEET', 'SRC_TOTAL', 'STORYFACTCHECKSTWEET', 'SENSATIONALISM', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES']]
#unit of analysis is story
dfstory=df[['STORYNUMBER', 'CODER', 'YEAR', 'PUBLICATIONDATE', 'URL', 'PUBLISHER', "PUBLISHERYEAR",
       'STORYTYPE', 'STORYTOPIC', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES',
       'STORYCONTAINSFALSESTATEMENT', 'STORYCLAIMSANOTHERSOURCEISFALSE',
       'STORYFACTCHECKSTWEET', 'SENSATIONALISM', 'src_expert', 'src_opponent',
       'src_otherpolitician', 'src_privatecitizen', 'src_media', 'src_other', "TWEETROLETOTHESTORY", "STORYROLETOTWEET", "SRC_TOTAL", "TWEETBY"]]
#remove duplicates
dfstory.drop_duplicates(subset=['STORYNUMBER'], keep='first', inplace=True)
#dv list for tweets and stories
DVtweet=['TWEETTOPIC','TWEETGENERICFRAMESDOMINANT', 'TWEETEMOTIONALFRAMES',
       #  'TWEETCONTAINSFALSESTATEMENT', 'TWEETCLAIMSSOMETHINGELSEISFALSE'
        ]
DVstory=['STORYTOPIC', 'STORYGENERICFRAMESDOMINANT', 'STORYEMOTIONALFRAMES',
       # 'STORYCONTAINSFALSESTATEMENT', 'STORYCLAIMSANOTHERSOURCEISFALSE'
       ]
DVnamesTweet=['the topic of the tweet','the generic frames of the tweets','the emotional frames of the tweets']
DVnamesStory=['the topic of the story','the generic frames of the stories','the emotional frames of the stories']
#list of different datasets
dftweet2020=dftweet.loc[dftweet['YEAR'] == 2020]
dftweet2016=dftweet.loc[dftweet['YEAR'] == 2016]
dftweetFOX=dftweet.loc[dftweet['PUBLISHER'] == "fox"]
dftweetCNN=dftweet.loc[dftweet['PUBLISHER'] == "cnn"]
dftweetFOX2016=dftweetFOX.loc[dftweetFOX['YEAR'] == 2016]
dftweetCNN2016=dftweetCNN.loc[dftweetCNN['YEAR'] == 2016]
dftweetFOX2020=dftweetFOX.loc[dftweetFOX['YEAR'] == 2020]
dftweetCNN2020=dftweetCNN.loc[dftweetCNN['YEAR'] == 2020]
dfstory2020=dfstory.loc[dfstory['YEAR'] == 2020]
dfstory2016=dfstory.loc[dfstory['YEAR'] == 2016]
dfstoryFOX=dfstory.loc[dfstory['PUBLISHER'] == "fox"]
dfstoryCNN=dfstory.loc[dfstory['PUBLISHER'] == "cnn"]
dfstoryFOX2016=dfstoryFOX.loc[dfstoryFOX['YEAR'] == 2016]
dfstoryCNN2016=dfstoryCNN.loc[dfstoryCNN['YEAR'] == 2016]
dfstoryFOX2020=dfstoryFOX.loc[dfstoryFOX['YEAR'] == 2020]
dfstoryCNN2020=dfstoryCNN.loc[dfstoryCNN['YEAR'] == 2020]
dftweetPres=dftweet.loc[dftweet['TWEETBY'].isin(['trump', 'biden'])]
dfstoryPres=dfstory.loc[dfstory['TWEETBY'].isin(['trump', 'biden'])]
dftweetPres2016=dftweetPres.loc[dftweetPres['YEAR'] == 2016]
dftweetPres2020=dftweetPres.loc[dftweetPres['YEAR'] == 2020]
dfstoryPres2016=dfstoryPres.loc[dfstoryPres['YEAR'] == 2016]
dfstoryPres2020=dfstoryPres.loc[dfstoryPres['YEAR'] == 2020]
pubyear=['cnn2016','cnn2020','fox2016','fox2020']

In [32]:
rq1_1=pd.crosstab(dfstory['STORYGENERICFRAMESDOMINANT'], dfstory['PUBLISHERYEAR'], margins=True)# crosstab with raw numbers
rq1_1.reset_index(inplace=True)
rq1_1.columns.name = None# remove the name of the columns
rq1_1p=pd.crosstab(dfstory['STORYGENERICFRAMESDOMINANT'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100# crosstab with percentages
rq1_1p.reset_index(inplace=True)
rq1_1p.columns.name = None
rq1_1a=pd.merge(rq1_1, rq1_1p, on="STORYGENERICFRAMESDOMINANT")#merge the two tables
rq1_1a=rq1_1a[['STORYGENERICFRAMESDOMINANT', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]#reorder columns
rq1_1a["All_p"]=rq1_1a["All"]/rq1_1a.All.sum()*100
rq1_1a["All_p"]=rq1_1a["All_p"].round(2)
rq1_1a.loc["All"] = rq1_1a.sum()
rq1_1a.to_csv('rq1_1a.csv')#export to csv
rq1_1a.set_index('STORYGENERICFRAMESDOMINANT', inplace=True)#set index
rq1_1a

,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
STORYGENERICFRAMESDOMINANT,,,,,,,,,,
conflict,55,85.94,91,76.47,15,83.33,56,73.68,217,78.34
economic,3,4.69,11,9.24,1,5.56,0,0.00,15,5.42
human interest,3,4.69,12,10.08,1,5.56,17,22.37,33,11.91
morality,1,1.56,0,0.00,1,5.56,0,0.00,2,0.72
responsibility,2,3.12,5,4.20,0,0.00,3,3.95,10,3.61
conflicteconomichuman interestmoralityresponsibility,64,100.00,119,99.99,18,100.01,76,100.00,277,100.00


In [33]:
print('STORYGENERICFRAMESDOMINANT')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYGENERICFRAMESDOMINANT'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYGENERICFRAMESDOMINANT'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYGENERICFRAMESDOMINANT'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYGENERICFRAMESDOMINANT'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))


STORYGENERICFRAMESDOMINANT
CNNvFox2016: chi2 = 1.54, p = 0.82
CNNvFox2020: chi2 = 11.79, p = 0.01
CNN2016v2020: chi2 = 5.06, p = 0.28
FOX2016v2020: chi2 = 11.48, p = 0.02


In [34]:
# #chisquaers
# pubyearA=['cnn2016_x','fox2016_x','cnn2016_x','cnn2020_x']
# pubyearB=['cnn2020_x','fox2020_x','fox2016_x','fox2020_x']
# tweetbyA=['trump_x','trump_x','trump_x','biden_x', 'biden_x','harris_x']
# tweetbyB=['biden_x','pence_x','harris_x','pence_x', 'harris_x','pence_x']
# pub=[rq1_1a, rq1_2a, rq2_1a, rq2_2a, rq3_1a, rq3_2a, rq3_3a, rq5_1a, rq5_2a, rq5_3a]
# candidate=[rq3_4a, rq3_5a, rq3_6a, rq3_7a]
# sig=" was significant (CHI2 = "
# nosig=" was not significant (CHI2 = "

# for i, j in zip(pubyearA, pubyearB):
#     expected, observed, stats = pg.chi2_independence(rq1_1a, x=i,y=j)
#     hi2=stats.iloc[0,2]
#     pval=stats.iloc[0,4]
#     if pval<0.05:
#         print("The relationship between " + i + " and " + j + " for " + rq1_1a.columns[0] + sig + chi2.round(2).astype(str) + ", p = "+pval.round(2).astype(str)+").")
#     else:
#         print("The relationship between " + i + " and " + j+ " for " + rq1_1a.columns[0] + nosig + chi2.round(2).astype(str) + ", p = "+pval.round(2).astype(str)+").")

In [35]:
rq1_2=pd.crosstab(dfstory['STORYEMOTIONALFRAMES'], dfstory['PUBLISHERYEAR'], margins=True)
rq1_2.reset_index(inplace=True)
rq1_2.columns.name = None
rq1_2p=pd.crosstab(dfstory['STORYEMOTIONALFRAMES'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq1_2p.reset_index(inplace=True)
rq1_2p.columns.name = None
rq1_2a=pd.merge(rq1_2, rq1_2p, on="STORYEMOTIONALFRAMES")
rq1_2a=rq1_2a[['STORYEMOTIONALFRAMES', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq1_2a["All_p"]=rq1_2a["All"]/rq1_2a.All.sum()*100
rq1_2a["All_p"]=rq1_2a["All_p"].round(2)
rq1_2a.loc["All"] = rq1_2a.sum()
rq1_2a.to_csv('rq1_2a.csv')
rq1_2a

,STORYEMOTIONALFRAMES,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,negative,60,93.75,108,90.76,16,84.21,55,72.37,239,85.97
1,neutral,3,4.69,0,0.00,2,10.53,0,0.00,5,1.80
2,positive,1,1.56,11,9.24,1,5.26,21,27.63,34,12.23
All,negativeneutralpositive,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00


In [36]:
print("STORYEMOTIONALFRAMES")
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYEMOTIONALFRAMES'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYEMOTIONALFRAMES'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYEMOTIONALFRAMES'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYEMOTIONALFRAMES'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYEMOTIONALFRAMES
CNNvFox2016: chi2 = 1.81, p = 0.41
CNNvFox2020: chi2 = 10.13, p = 0.0
CNN2016v2020: chi2 = 9.36, p = 0.01
FOX2016v2020: chi2 = 11.57, p = 0.0


RQ2: What are the frames of the candidates’ tweets selected by journalists for inclusion in news stories?
-	Data analysis of frames in candidates tweets
-	Presentation of tweets in news stories 
-	2016 and 2020

Create a table like this
             	Trump2016	Pence	Hillary  Kaine Trump2020   Pence2020   Biden    Harris
Frame 1			%	%		%		%
Frame 2
Frame 3
And so forth.....
Positive	
Negative


In [37]:
rq2_1=pd.crosstab(dftweet['TWEETGENERICFRAMESDOMINANT'], dftweet['PUBLISHERYEAR'], margins=True)
rq2_1.reset_index(inplace=True)
rq2_1.columns.name = None
rq2_1p=pd.crosstab(dftweet['TWEETGENERICFRAMESDOMINANT'], dftweet['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq2_1p.reset_index(inplace=True)
rq2_1p.columns.name = None
rq2_1a=pd.merge(rq2_1, rq2_1p, on="TWEETGENERICFRAMESDOMINANT")
rq2_1a=rq2_1a[['TWEETGENERICFRAMESDOMINANT', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq2_1a["All_p"]=rq2_1a["All"]/rq2_1a.All.sum()*100
rq2_1a["All_p"]=rq2_1a["All_p"].round(2)
rq2_1a.loc["All"] = rq2_1a.sum()
rq2_1a.to_csv('rq2_1a.csv')
rq2_1a

,TWEETGENERICFRAMESDOMINANT,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,conflict,94,76.42,73,51.77,20,58.82,73,60.83,260,62.20
1,economic,1,0.81,16,11.35,1,2.94,3,2.50,21,5.02
2,human interest,14,11.38,36,25.53,13,38.24,32,26.67,95,22.73
3,morality,4,3.25,9,6.38,0,0.00,3,2.50,16,3.83
4,responsibility,10,8.13,7,4.96,0,0.00,9,7.50,26,6.22
All,conflicteconomichuman interestmoralityresponsi...,123,99.99,141,99.99,34,100.00,120,100.00,418,100.00


In [38]:
print("TWEETGENERICFRAMESDOMINANT")
#CNNvFox2016
crosstab=pd.crosstab(dftweet2016['TWEETGENERICFRAMESDOMINANT'], dftweet2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dftweet2020['TWEETGENERICFRAMESDOMINANT'], dftweet2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dftweetCNN['TWEETGENERICFRAMESDOMINANT'], dftweetCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dftweetFOX['TWEETGENERICFRAMESDOMINANT'], dftweetFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

TWEETGENERICFRAMESDOMINANT
CNNvFox2016: chi2 = 17.12, p = 0.0
CNNvFox2020: chi2 = 10.76, p = 0.03
CNN2016v2020: chi2 = 26.91, p = 0.0
FOX2016v2020: chi2 = 4.65, p = 0.33


In [39]:
rq2_2=pd.crosstab(dftweet['TWEETEMOTIONALFRAMES'], dftweet['PUBLISHERYEAR'], margins=True)
rq2_2.reset_index(inplace=True)
rq2_2.columns.name = None
rq2_2p=pd.crosstab(dftweet['TWEETEMOTIONALFRAMES'], dftweet['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq2_2p.reset_index(inplace=True)
rq2_2p.columns.name = None
rq2_2a=pd.merge(rq2_2, rq2_2p, on="TWEETEMOTIONALFRAMES")
rq2_2a=rq2_2a[['TWEETEMOTIONALFRAMES', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq2_2a["All_p"]=rq2_2a["All"]/rq2_2a.All.sum()*100
rq2_2a["All_p"]=rq2_2a["All_p"].round(2)
rq2_2a.loc["All"] = rq2_2a.sum()
rq2_2a.to_csv('rq2_2a.csv')
rq2_2a

,TWEETEMOTIONALFRAMES,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,negative,104,84.55,84,59.15,20,58.82,72,60.0,280,66.83
1,neutral,5,4.07,0,0.00,2,5.88,0,0.0,7,1.67
2,positive,14,11.38,58,40.85,12,35.29,48,40.0,132,31.50
All,negativeneutralpositive,123,100.00,142,100.00,34,99.99,120,100.0,419,100.00


In [40]:
print("TWEETEMOTIONALFRAMES")
#CNNvFox2016
crosstab=pd.crosstab(dftweet2016['TWEETEMOTIONALFRAMES'], dftweet2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dftweet2020['TWEETEMOTIONALFRAMES'], dftweet2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dftweetCNN['TWEETEMOTIONALFRAMES'], dftweetCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dftweetFOX['TWEETEMOTIONALFRAMES'], dftweetFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

TWEETEMOTIONALFRAMES
CNNvFox2016: chi2 = 11.63, p = 0.0
CNNvFox2020: chi2 = 0.0, p = 0.99
CNN2016v2020: chi2 = 32.82, p = 0.0
FOX2016v2020: chi2 = 7.22, p = 0.03


RQ3: How did CNN.com and FoxNews.com report Donald Trump’s tweets compared to the other candidates in the 2016 and 2020 presidential election campaign news stories? Are there any significant differences between the two campaign coverages?
-	Report that the majority of news stories using tweets were using Trump’s tweets
-	Report presentation of tweets (embedding, citing, etc.)
-	Report the dominant frame of Trump’s tweets and the dominant frame in Hillary’s and Biden’s tweets but without repeating too much from RQ1 and RQ2
-	Report fact-checking, sensationalism, number of sources, and role of story to tweets for Trump vs Hillary and Biden. 


In [41]:
# Report that the majority of news stories using tweets were using Trump’s tweets
rq3_1=pd.crosstab(dfstory['TWEETBY'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_1.reset_index(inplace=True)
rq3_1.columns.name = None
rq3_1p=pd.crosstab(dfstory['TWEETBY'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq3_1p.reset_index(inplace=True)
rq3_1p.columns.name = None
rq3_1a=pd.merge(rq3_1, rq3_1p, on="TWEETBY")
rq3_1a=rq3_1a[['TWEETBY', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq3_1a["All_p"]=rq3_1a["All"]/rq3_1a.All.sum()*100
rq3_1a["All_p"]=rq3_1a["All_p"].round(2)
rq3_1a.loc["All"] = rq3_1a.sum()
rq3_1a.to_csv('rq3_1a.csv')
rq3_1a

,TWEETBY,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,biden,10,15.62,5,4.20,1,5.26,6,7.89,22,7.91
1,harris,0,0.00,6,5.04,0,0.00,4,5.26,10,3.60
2,pence,3,4.69,1,0.84,2,10.53,0,0.00,6,2.16
3,trump,51,79.69,107,89.92,16,84.21,66,86.84,240,86.33
All,bidenharrispencetrump,64,100.00,119,100.00,19,100.00,76,99.99,278,100.00


In [42]:
print('TWEETBY')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['TWEETBY'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['TWEETBY'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['TWEETBY'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['TWEETBY'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))


TWEETBY
CNNvFox2016: chi2 = 2.05, p = 0.36
CNNvFox2020: chi2 = 1.81, p = 0.61
CNN2016v2020: chi2 = 13.17, p = 0.0
FOX2016v2020: chi2 = 9.16, p = 0.03


In [43]:
# Report presentation of tweets (embedding, citing, etc.)
rq3_2=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_2.reset_index(inplace=True)
rq3_2.columns.name = None
rq3_2p=pd.crosstab(dfstory['TWEETROLETOTHESTORY'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq3_2p.reset_index(inplace=True)
rq3_2p.columns.name = None
rq3_2a=pd.merge(rq3_2, rq3_2p, on="TWEETROLETOTHESTORY")
rq3_2a=rq3_2a[['TWEETROLETOTHESTORY', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq3_2a["All_p"]=rq3_2a["All"]/rq3_2a.All.sum()*100
rq3_2a["All_p"]=rq3_2a["All_p"].round(2)
rq3_2a.loc["All"] = rq3_2a.sum()
rq3_2a.to_csv('rq3_2a.csv')
rq3_2a


,TWEETROLETOTHESTORY,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,illustration,34,53.12,36,30.25,8,42.11,24,31.58,102,36.69
1,non-primary source,15,23.44,33,27.73,9,47.37,3,3.95,60,21.58
2,trigger,15,23.44,50,42.02,2,10.53,49,64.47,116,41.73
All,illustrationnon-primary sourcetrigger,64,100.00,119,100.00,19,100.01,76,100.00,278,100.00


In [44]:
print('TWEETROLETOTHESTORY')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['TWEETROLETOTHESTORY'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['TWEETROLETOTHESTORY'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['TWEETROLETOTHESTORY'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['TWEETROLETOTHESTORY'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))


TWEETROLETOTHESTORY
CNNvFox2016: chi2 = 4.45, p = 0.11
CNNvFox2020: chi2 = 18.84, p = 0.0
CNN2016v2020: chi2 = 10.03, p = 0.01
FOX2016v2020: chi2 = 31.43, p = 0.0


In [45]:
rq3_3=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['PUBLISHERYEAR'], margins=True)
rq3_3.reset_index(inplace=True)
rq3_3.columns.name = None
rq3_3p=pd.crosstab(dfstory['STORYROLETOTWEET'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq3_3p.reset_index(inplace=True)
rq3_3p.columns.name = None
rq3_3a=pd.merge(rq3_3, rq3_3p, on="STORYROLETOTWEET")
rq3_3a=rq3_3a[['STORYROLETOTWEET', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq3_3a["All_p"]=rq3_3a["All"]/rq3_3a.All.sum()*100
rq3_3a["All_p"]=rq3_3a["All_p"].round(2)
rq3_3a.loc["All"] = rq3_3a.sum()
rq3_3a.to_csv('rq3_3a.csv')
rq3_3a

,STORYROLETOTWEET,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,amplifies the tweet,8,12.50,4,3.36,0,0.00,49,64.47,61,21.94
1,ctiticizes the tweet,34,53.12,59,49.58,1,5.26,4,5.26,98,35.25
2,neutral,22,34.38,56,47.06,18,94.74,23,30.26,119,42.81
All,amplifies the tweetctiticizes the tweetneutral,64,100.00,119,100.00,19,100.00,76,99.99,278,100.00


In [46]:
print('STORYROLETOTWEET')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYROLETOTWEET'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYROLETOTWEET'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYROLETOTWEET'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYROLETOTWEET'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYROLETOTWEET
CNNvFox2016: chi2 = 21.41, p = 0.0
CNNvFox2020: chi2 = 95.15, p = 0.0
CNN2016v2020: chi2 = 6.97, p = 0.03
FOX2016v2020: chi2 = 26.89, p = 0.0


In [47]:
# Report the dominant frame of Trump’s tweets and the dominant frame in Hillary’s and Biden’s tweets but without repeating too much from RQ1 and RQ2
# Report stories fact-checking tweets for Trump vs Hillary, 2016
rq3_a=pd.crosstab(dftweetPres2016['STORYGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], margins=True)
rq3_a.reset_index(inplace=True)
rq3_a.columns.name = None
rq3_ap=pd.crosstab(dftweetPres2016['STORYGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_ap.reset_index(inplace=True)
rq3_ap.columns.name = None
rq3_aa=pd.merge(rq3_a, rq3_ap, on="STORYGENERICFRAMESDOMINANT")
rq3_aa=rq3_aa[['STORYGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_aa["All_p"]=rq3_aa["All"]/rq3_aa.All.sum()*100
rq3_aa["All_p"]=rq3_aa["All_p"].round(2)
rq3_aa.loc["All"] = rq3_aa.sum()
rq3_aa.to_csv('rq3_aa.csv')
rq3_aa

,STORYGENERICFRAMESDOMINANT,trump_x,trump_y,biden_x,biden_y,All,All_p
0,conflict,117,93.6,11,57.89,128,88.89
1,economic,4,3.2,3,15.79,7,4.86
2,human interest,0,0.0,4,21.05,4,2.78
3,morality,1,0.8,1,5.26,2,1.39
4,responsibility,3,2.4,0,0.00,3,2.08
All,conflicteconomichuman interestmoralityresponsi...,125,100.0,19,99.99,144,100.00


In [48]:
rq3_a2=pd.crosstab(dftweetPres2020['STORYGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], margins=True)
rq3_a2.reset_index(inplace=True)
rq3_a2.columns.name = None
rq3_a2p=pd.crosstab(dftweetPres2020['STORYGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_a2p.reset_index(inplace=True)
rq3_a2p.columns.name = None
rq3_a2a=pd.merge(rq3_a2, rq3_a2p, on="STORYGENERICFRAMESDOMINANT")
rq3_a2a=rq3_a2a[['STORYGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_a2a["All_p"]=rq3_a2a["All"]/rq3_a2a.All.sum()*100
rq3_a2a["All_p"]=rq3_a2a["All_p"].round(2)
rq3_a2a.loc["All"] = rq3_a2a.sum()
rq3_a2a.to_csv('rq3_a2a.csv')
rq3_a2a

,STORYGENERICFRAMESDOMINANT,trump_x,trump_y,biden_x,biden_y,All,All_p
0,conflict,170,76.92,19,76.0,189,76.83
1,economic,13,5.88,0,0.0,13,5.28
2,human interest,32,14.48,4,16.0,36,14.63
3,responsibility,6,2.71,2,8.0,8,3.25
All,conflicteconomichuman interestresponsibility,221,99.99,25,100.0,246,99.99


In [49]:
rq3_b=pd.crosstab(dftweetPres2016['TWEETGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], margins=True)
rq3_b.reset_index(inplace=True)
rq3_b.columns.name = None
rq3_bp=pd.crosstab(dftweetPres2016['TWEETGENERICFRAMESDOMINANT'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_bp.reset_index(inplace=True)
rq3_bp.columns.name = None
rq3_ba=pd.merge(rq3_b, rq3_bp, on="TWEETGENERICFRAMESDOMINANT")
rq3_ba=rq3_ba[['TWEETGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_ba["All_p"]=rq3_ba["All"]/rq3_ba.All.sum()*100
rq3_ba["All_p"]=rq3_ba["All_p"].round(2)
rq3_ba.loc["All"] = rq3_ba.sum()
rq3_ba.to_csv('rq3_ba.csv')
rq3_ba

,TWEETGENERICFRAMESDOMINANT,trump_x,trump_y,biden_x,biden_y,All,All_p
0,conflict,103,78.63,8,42.11,111,74.00
1,economic,1,0.76,1,5.26,2,1.33
2,human interest,16,12.21,7,36.84,23,15.33
3,morality,3,2.29,1,5.26,4,2.67
4,responsibility,8,6.11,2,10.53,10,6.67
All,conflicteconomichuman interestmoralityresponsi...,131,100.00,19,100.00,150,100.00


In [50]:
rq3_b2=pd.crosstab(dftweetPres2020['TWEETGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], margins=True)
rq3_b2.reset_index(inplace=True)
rq3_b2.columns.name = None
rq3_b2p=pd.crosstab(dftweetPres2020['TWEETGENERICFRAMESDOMINANT'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_b2p.reset_index(inplace=True)
rq3_b2p.columns.name = None
rq3_b2a=pd.merge(rq3_b, rq3_b2p, on="TWEETGENERICFRAMESDOMINANT")
rq3_b2a=rq3_b2a[['TWEETGENERICFRAMESDOMINANT', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_b2a["All_p"]=rq3_b2a["All"]/rq3_b2a.All.sum()*100
rq3_b2a["All_p"]=rq3_b2a["All_p"].round(2)
rq3_b2a.loc["All"] = rq3_b2a.sum()
rq3_b2a.to_csv('rq3_b2a.csv')
rq3_b2a

,TWEETGENERICFRAMESDOMINANT,trump_x,trump_y,biden_x,biden_y,All,All_p
0,conflict,103,60.00,8,48.0,111,74.00
1,economic,1,7.73,1,4.0,2,1.33
2,human interest,16,26.82,7,20.0,23,15.33
3,morality,3,1.82,1,16.0,4,2.67
4,responsibility,8,3.64,2,12.0,10,6.67
All,conflicteconomichuman interestmoralityresponsi...,131,100.01,19,100.0,150,100.00


In [51]:
rq3_c=pd.crosstab(dftweetPres2016['STORYEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], margins=True)
rq3_c.reset_index(inplace=True)
rq3_c.columns.name = None
rq3_cp=pd.crosstab(dftweetPres2016['STORYEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_cp.reset_index(inplace=True)
rq3_cp.columns.name = None
rq3_ca=pd.merge(rq3_c, rq3_cp, on="STORYEMOTIONALFRAMES")
rq3_ca=rq3_ca[['STORYEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_ca["All_p"]=rq3_ca["All"]/rq3_ca.All.sum()*100
rq3_ca["All_p"]=rq3_ca["All_p"].round(2)
rq3_ca.loc["All"] = rq3_ca.sum()
rq3_ca.to_csv('rq3_ca.csv')
rq3_ca

,STORYEMOTIONALFRAMES,trump_x,trump_y,biden_x,biden_y,All,All_p
0,negative,129,98.47,14,73.68,143,95.33
1,neutral,1,0.76,4,21.05,5,3.33
2,positive,1,0.76,1,5.26,2,1.33
All,negativeneutralpositive,131,99.99,19,99.99,150,99.99


In [53]:
rq3_c2=pd.crosstab(dftweetPres2020['STORYEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'], margins=True)
rq3_c2.reset_index(inplace=True)
rq3_c2.columns.name = None
rq3_c2p=pd.crosstab(dftweetPres2020['STORYEMOTIONALFRAMES'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_c2p.reset_index(inplace=True)
rq3_c2p.columns.name = None
rq3_c2a=pd.merge(rq3_c2, rq3_c2p, on="STORYEMOTIONALFRAMES")
rq3_c2a=rq3_c2a[['STORYEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_c2a["All_p"]=rq3_c2a["All"]/rq3_c2a.All.sum()*100
rq3_c2a["All_p"]=rq3_c2a["All_p"].round(2)
rq3_c2a.loc["All"] = rq3_c2a.sum()
rq3_c2a.to_csv('rq3_c2a.csv')
rq3_c2a

,STORYEMOTIONALFRAMES,trump_x,trump_y,biden_x,biden_y,All,All_p
0,negative,185,83.71,20,80.0,205,83.33
1,positive,36,16.29,5,20.0,41,16.67
All,negativepositive,221,100.00,25,100.0,246,100.00


In [55]:
rq3_d=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], margins=True)
rq3_d.reset_index(inplace=True)
rq3_d.columns.name = None
rq3_dp=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_dp.reset_index(inplace=True)
rq3_dp.columns.name = None
rq3_da=pd.merge(rq3_d, rq3_dp, on="TWEETEMOTIONALFRAMES")
rq3_da=rq3_da[['TWEETEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_da["All_p"]=rq3_da["All"]/rq3_da.All.sum()*100
rq3_da["All_p"]=rq3_da["All_p"].round(2)
rq3_da.loc["All"] = rq3_da.sum()
rq3_da.to_csv('rq3_da.csv')
rq3_da

,TWEETEMOTIONALFRAMES,trump_x,trump_y,biden_x,biden_y,All,All_p
0,negative,113,86.26,9,47.37,122,81.33
1,neutral,2,1.53,4,21.05,6,4.00
2,positive,16,12.21,6,31.58,22,14.67
All,negativeneutralpositive,131,100.00,19,100.00,150,100.00


In [56]:
rq3_d2=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], margins=True)
rq3_d2.reset_index(inplace=True)
rq3_d2.columns.name = None
rq3_d2p=pd.crosstab(dftweetPres2016['TWEETEMOTIONALFRAMES'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_d2p.reset_index(inplace=True)
rq3_d2p.columns.name = None
rq3_d2a=pd.merge(rq3_d2, rq3_d2p, on="TWEETEMOTIONALFRAMES")
rq3_d2a=rq3_d2a[['TWEETEMOTIONALFRAMES', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_d2a["All_p"]=rq3_d2a["All"]/rq3_d2a.All.sum()*100
rq3_d2a["All_p"]=rq3_d2a["All_p"].round(2)
rq3_d2a.loc["All"] = rq3_d2a.sum()
rq3_d2a.to_csv('rq3_d2a.csv')
rq3_d2a

,TWEETEMOTIONALFRAMES,trump_x,trump_y,biden_x,biden_y,All,All_p
0,negative,113,86.26,9,47.37,122,81.33
1,neutral,2,1.53,4,21.05,6,4.00
2,positive,16,12.21,6,31.58,22,14.67
All,negativeneutralpositive,131,100.00,19,100.00,150,100.00


In [ ]:
# Report stories fact-checking tweets for Trump vs Hillary, 2016
rq3_4=pd.crosstab(dftweetPres2016['STORYFACTCHECKSTWEET'], dftweetPres2016['TWEETBY'], margins=True)
rq3_4.reset_index(inplace=True)
rq3_4.columns.name = None
rq3_4p=pd.crosstab(dftweetPres2016['STORYFACTCHECKSTWEET'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_4p.reset_index(inplace=True)
rq3_4p.columns.name = None
rq3_4a=pd.merge(rq3_4, rq3_4p, on="STORYFACTCHECKSTWEET")
rq3_4a=rq3_4a[['STORYFACTCHECKSTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_4a["All_p"]=rq3_4a["All"]/rq3_4a.All.sum()*100
rq3_4a["All_p"]=rq3_4a["All_p"].round(2)
rq3_4a.loc["All"] = rq3_4a.sum()
rq3_4a.to_csv('rq3_4a.csv')
rq3_4a

,STORYFACTCHECKSTWEET,trump_x,trump_y,biden_x,biden_y,All,All_p
0,no,79,60.31,17,89.47,96,64.0
1,yes,52,39.69,2,10.53,54,36.0
All,noyes,131,100.00,19,100.00,150,100.0


In [ ]:
# Report stories fact-checking tweets for Trump vs  vs  Biden, 2020
rq3_4_2=pd.crosstab(dftweetPres2020['STORYFACTCHECKSTWEET'], dftweetPres2020['TWEETBY'], margins=True)
rq3_4_2.reset_index(inplace=True)
rq3_4_2.columns.name = None
rq3_4_2p=pd.crosstab(dftweetPres2020['STORYFACTCHECKSTWEET'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_4_2p.reset_index(inplace=True)
rq3_4_2p.columns.name = None
rq3_4_2a=pd.merge(rq3_4_2, rq3_4_2p, on="STORYFACTCHECKSTWEET")
rq3_4_2a=rq3_4_2a[['STORYFACTCHECKSTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_4_2a["All_p"]=rq3_4_2a["All"]/rq3_4_2a.All.sum()*100
rq3_4_2a["All_p"]=rq3_4_2a["All_p"].round(2)
rq3_4_2a.loc["All"] = rq3_4_2a.sum()
rq3_4_2a.to_csv('rq3_4_2a.csv')
rq3_4_2a

,STORYFACTCHECKSTWEET,trump_x,trump_y,biden_x,biden_y,All,All_p
0,no,161,72.85,24,96.0,185,75.2
1,yes,60,27.15,1,4.0,61,24.8
All,noyes,221,100.00,25,100.0,246,100.0


In [ ]:
# Report stories sensationalism tweets for Trump vs Hillary, 2016
rq3_5=pd.crosstab(dftweetPres2016['SENSATIONALISM'], dftweetPres2016['TWEETBY'], margins=True)
rq3_5.reset_index(inplace=True)
rq3_5.columns.name = None
rq3_5p=pd.crosstab(dftweetPres2016['SENSATIONALISM'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_5p.reset_index(inplace=True)
rq3_5p.columns.name = None
rq3_5a=pd.merge(rq3_5, rq3_5p, on="SENSATIONALISM")
rq3_5a=rq3_5a[['SENSATIONALISM', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_5a["All_p"]=rq3_5a["All"]/rq3_5a.All.sum()*100
rq3_5a["All_p"]=rq3_5a["All_p"].round(2)
rq3_5a.loc["All"] = rq3_5a.sum()
rq3_5a.to_csv('rq3_5a.csv')
rq3_5a

,SENSATIONALISM,trump_x,trump_y,biden_x,biden_y,All,All_p
0,no,91,69.47,16,88.89,107,71.81
1,yes,40,30.53,2,11.11,42,28.19
All,noyes,131,100.00,18,100.00,149,100.00


In [ ]:
# Report stories sensationalism tweets for Trump vs  vs  Biden, 2020
rq3_5_2=pd.crosstab(dftweetPres2020['SENSATIONALISM'], dftweetPres2020['TWEETBY'], margins=True)
rq3_5_2.reset_index(inplace=True)
rq3_5_2.columns.name = None
rq3_5_2p=pd.crosstab(dftweetPres2020['SENSATIONALISM'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_5_2p.reset_index(inplace=True)
rq3_5_2p.columns.name = None
rq3_5_2a=pd.merge(rq3_5, rq3_5p, on="SENSATIONALISM")
rq3_5_2a=rq3_5_2a[['SENSATIONALISM', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_5_2a["All_p"]=rq3_5_2a["All"]/rq3_5_2a.All.sum()*100
rq3_5_2a["All_p"]=rq3_5_2a["All_p"].round(2)
rq3_5_2a.loc["All"] = rq3_5_2a.sum()
rq3_5_2a.to_csv('rq3_5_2a.csv')
rq3_5_2a

,SENSATIONALISM,trump_x,trump_y,biden_x,biden_y,All,All_p
0,no,91,69.47,16,88.89,107,71.81
1,yes,40,30.53,2,11.11,42,28.19
All,noyes,131,100.00,18,100.00,149,100.00


In [ ]:
# Report stories tweet's role to stories for Trump vs Hillary, 2016
rq3_6=pd.crosstab(dftweetPres2016['TWEETROLETOTHESTORY'], dftweetPres2016['TWEETBY'], margins=True)
rq3_6.reset_index(inplace=True)
rq3_6.columns.name = None
rq3_6p=pd.crosstab(dftweetPres2016['TWEETROLETOTHESTORY'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_6p.reset_index(inplace=True)
rq3_6p.columns.name = None
rq3_6a=pd.merge(rq3_6, rq3_6p, on="TWEETROLETOTHESTORY")
rq3_6a=rq3_6a[['TWEETROLETOTHESTORY', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_6a["All_p"]=rq3_6a["All"]/rq3_6a.All.sum()*100
rq3_6a["All_p"]=rq3_6a["All_p"].round(2)
rq3_6a.loc["All"] = rq3_6a.sum()
rq3_6a.to_csv('rq3_6a.csv')
rq3_6a


,TWEETROLETOTHESTORY,trump_x,trump_y,biden_x,biden_y,All,All_p
0,illustration,56,42.75,9,47.37,65,43.33
1,non-primary source,24,18.32,8,42.11,32,21.33
2,trigger,51,38.93,2,10.53,53,35.33
All,illustrationnon-primary sourcetrigger,131,100.00,19,100.01,150,99.99


In [ ]:
# Report stories tweet's role to stories for Trump vs  vs  Biden, 2020
rq3_6_2=pd.crosstab(dftweetPres2020['TWEETROLETOTHESTORY'], dftweetPres2020['TWEETBY'], margins=True)
rq3_6_2.reset_index(inplace=True)
rq3_6_2.columns.name = None
rq3_6_2p=pd.crosstab(dftweetPres2020['TWEETROLETOTHESTORY'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_6_2p.reset_index(inplace=True)
rq3_6_2p.columns.name = None
rq3_6_2a=pd.merge(rq3_6, rq3_6p, on="TWEETROLETOTHESTORY")
rq3_6_2a=rq3_6_2a[['TWEETROLETOTHESTORY', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_6_2a["All_p"]=rq3_6_2a["All"]/rq3_6_2a.All.sum()*100
rq3_6_2a["All_p"]=rq3_6_2a["All_p"].round(2)
rq3_6_2a.loc["All"] = rq3_6_2a.sum()
rq3_6_2a.to_csv('rq3_6_2a.csv')
rq3_6_2a


,TWEETROLETOTHESTORY,trump_x,trump_y,biden_x,biden_y,All,All_p
0,illustration,56,42.75,9,47.37,65,43.33
1,non-primary source,24,18.32,8,42.11,32,21.33
2,trigger,51,38.93,2,10.53,53,35.33
All,illustrationnon-primary sourcetrigger,131,100.00,19,100.01,150,99.99


In [ ]:
# Report stories story's role to tweets for Trump vs Hillary, 2016
rq3_7=pd.crosstab(dftweetPres2016['STORYROLETOTWEET'], dftweetPres2016['TWEETBY'], margins=True)
rq3_7.reset_index(inplace=True)
rq3_7.columns.name = None
rq3_7p=pd.crosstab(dftweetPres2016['STORYROLETOTWEET'], dftweetPres2016['TWEETBY'], normalize='columns').round(4)*100
rq3_7p.reset_index(inplace=True)
rq3_7p.columns.name = None
rq3_7a=pd.merge(rq3_7, rq3_7p, on="STORYROLETOTWEET")
rq3_7a=rq3_7a[['STORYROLETOTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_7a["All_p"]=rq3_7a["All"]/rq3_7a.All.sum()*100
rq3_7a["All_p"]=rq3_7a["All_p"].round(2)
rq3_7a.loc["All"] = rq3_7a.sum()
rq3_7a.to_csv('rq3_7a.csv')
rq3_7a

,STORYROLETOTWEET,trump_x,trump_y,biden_x,biden_y,All,All_p
0,amplifies the tweet,22,16.79,4,21.05,26,17.33
1,ctiticizes the tweet,65,49.62,1,5.26,66,44.00
2,neutral,44,33.59,14,73.68,58,38.67
All,amplifies the tweetctiticizes the tweetneutral,131,100.00,19,99.99,150,100.00


In [ ]:
#report stories story's role to tweets for Trump vs  vs  Biden, 2020
rq3_7_2=pd.crosstab(dftweetPres2020['STORYROLETOTWEET'], dftweetPres2020['TWEETBY'], margins=True)
rq3_7_2.reset_index(inplace=True)
rq3_7_2.columns.name = None
rq3_7_2p=pd.crosstab(dftweetPres2020['STORYROLETOTWEET'], dftweetPres2020['TWEETBY'], normalize='columns').round(4)*100
rq3_7_2p.reset_index(inplace=True)
rq3_7_2p.columns.name = None
rq3_7_2a=pd.merge(rq3_7, rq3_7p, on="STORYROLETOTWEET")
rq3_7_2a=rq3_7_2a[['STORYROLETOTWEET', 'trump_x','trump_y', 'biden_x', 'biden_y','All']]
rq3_7_2a["All_p"]=rq3_7_2a["All"]/rq3_7_2a.All.sum()*100
rq3_7_2a["All_p"]=rq3_7_2a["All_p"].round(2)
rq3_7_2a.loc["All"] = rq3_7_2a.sum()
rq3_7_2a.to_csv('rq3_7_2a.csv')
rq3_7_2a

,STORYROLETOTWEET,trump_x,trump_y,biden_x,biden_y,All,All_p
0,amplifies the tweet,22,16.79,4,21.05,26,17.33
1,ctiticizes the tweet,65,49.62,1,5.26,66,44.00
2,neutral,44,33.59,14,73.68,58,38.67
All,amplifies the tweetctiticizes the tweetneutral,131,100.00,19,99.99,150,100.00


In [ ]:
#STORYFACTCHECKSTWEET
print('STORYFACTCHECKSTWEET2016')
crosstab=pd.crosstab(dftweetPres2016['STORYFACTCHECKSTWEET'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# SENSATIONALISM
print('SENSATIONALISM2016')
crosstab=pd.crosstab(dftweetPres2016['SENSATIONALISM'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYROLETOTWEET
print('STORYROLETOTWEET2016')
crosstab=pd.crosstab(dftweetPres2016['STORYROLETOTWEET'], dftweetPres2016['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYFACTCHECKSTWEET2016
STORYFACTCHECKSTWEET2016: chi2 = 4.93, p = 0.03
SENSATIONALISM2016
STORYFACTCHECKSTWEET2016: chi2 = 2.07, p = 0.15
STORYROLETOTWEET2016
STORYFACTCHECKSTWEET2016: chi2 = 14.49, p = 0.0


In [ ]:
#STORYFACTCHECKSTWEET
print('STORYFACTCHECKSTWEET2020')
crosstab=pd.crosstab(dftweetPres2020['STORYFACTCHECKSTWEET'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYFACTCHECKSTWEET2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# SENSATIONALISM
print('SENSATIONALISM2020')
crosstab=pd.crosstab(dftweetPres2020['SENSATIONALISM'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("SENSATIONALISM2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# STORYROLETOTWEET
print('STORYROLETOTWEET2020')
crosstab=pd.crosstab(dftweetPres2020['STORYROLETOTWEET'], dftweetPres2020['TWEETBY'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("STORYROLETOTWEET2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYFACTCHECKSTWEET2020
STORYFACTCHECKSTWEET2020: chi2 = 5.27, p = 0.02
SENSATIONALISM2020
SENSATIONALISM2020: chi2 = 11.57, p = 0.0
STORYROLETOTWEET2020
STORYROLETOTWEET2020: chi2 = 4.04, p = 0.13


In [ ]:
pg.anova(data=dfstoryPres2020, dv='SRC_TOTAL', between='TWEETBY', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,TWEETBY,0.187164,1,0.187164,0.143659,0.705112,0.000789
1,Within,237.117183,182,1.302842,NaN,NaN,NaN


In [ ]:
pg.pairwise_ttests(data=dfstoryPres2020, dv='SRC_TOTAL', between='TWEETBY', padjust='bonf', effsize='cohen')

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,TWEETBY,biden,trump,False,True,0.335141,10.984553,two-sided,0.743832,0.319,0.117857


RQ5: What journalistic values were used by CNN and FOX in the stories using candidates’ tweets as news stories?
-	Sensationalism.  (percent, other statistics, comparison between the 2 media organizations)
-	Fact-checking
-	Number of sources 
-	Role of story to tweets
-	Other values we coded for

In [ ]:
rq5_1=pd.crosstab(dfstory['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_1.reset_index(inplace=True)
rq5_1.columns.name = None
rq5_1p=pd.crosstab(dfstory['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq5_1p.reset_index(inplace=True)
rq5_1p.columns.name = None
rq5_1a=pd.merge(rq5_1, rq5_1p, on="STORYCLAIMSANOTHERSOURCEISFALSE")
rq5_1a=rq5_1a[['STORYCLAIMSANOTHERSOURCEISFALSE', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq5_1a["All_p"]=rq5_1a["All"]/rq5_1a.All.sum()*100
rq5_1a["All_p"]=rq5_1a["All_p"].round(2)
rq5_1a.loc["All"] = rq5_1a.sum()
rq5_1a.to_csv('rq5_1a.csv')
rq5_1a

,STORYCLAIMSANOTHERSOURCEISFALSE,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,no,42,65.62,77,64.71,17,89.47,55,72.37,191,68.71
1,yes,22,34.38,42,35.29,2,10.53,21,27.63,87,31.29
All,noyes,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00


In [ ]:
print('STORYCLAIMSANOTHERSOURCEISFALSE')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYCLAIMSANOTHERSOURCEISFALSE'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYCLAIMSANOTHERSOURCEISFALSE
CNNvFox2016: chi2 = 2.98, p = 0.08
CNNvFox2020: chi2 = 0.92, p = 0.34
CNN2016v2020: chi2 = 0.0, p = 1.0
FOX2016v2020: chi2 = 1.58, p = 0.21


In [ ]:
rq5_2=pd.crosstab(dfstory['STORYFACTCHECKSTWEET'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_2.reset_index(inplace=True)
rq5_2.columns.name = None
rq5_2p=pd.crosstab(dfstory['STORYFACTCHECKSTWEET'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq5_2p.reset_index(inplace=True)
rq5_2p.columns.name = None
rq5_2a=pd.merge(rq5_2, rq5_2p, on="STORYFACTCHECKSTWEET")
rq5_2a=rq5_2a[['STORYFACTCHECKSTWEET', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq5_2a["All_p"]=rq5_2a["All"]/rq5_2a.All.sum()*100
rq5_2a["All_p"]=rq5_2a["All_p"].round(2)
rq5_2a.loc["All"] = rq5_2a.sum()
rq5_2a.to_csv('rq5_2a.csv')
rq5_2a

,STORYFACTCHECKSTWEET,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,no,43,67.19,80,67.23,18,94.74,68,89.47,209,75.18
1,yes,21,32.81,39,32.77,1,5.26,8,10.53,69,24.82
All,noyes,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00


In [ ]:
print('STORYFACTCHECKSTWEET')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['STORYFACTCHECKSTWEET'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['STORYFACTCHECKSTWEET'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['STORYFACTCHECKSTWEET'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['STORYFACTCHECKSTWEET'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

STORYFACTCHECKSTWEET
CNNvFox2016: chi2 = 4.38, p = 0.04
CNNvFox2020: chi2 = 11.36, p = 0.0
CNN2016v2020: chi2 = 0.0, p = 1.0
FOX2016v2020: chi2 = 0.07, p = 0.79


In [ ]:
rq5_3=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['PUBLISHERYEAR'], margins=True)
rq5_3.reset_index(inplace=True)
rq5_3.columns.name = None
rq5_3p=pd.crosstab(dfstory['SENSATIONALISM'], dfstory['PUBLISHERYEAR'], normalize='columns').round(4)*100
rq5_3p.reset_index(inplace=True)
rq5_3p.columns.name = None
rq5_3a=pd.merge(rq5_3, rq5_3p, on="SENSATIONALISM")
rq5_3a=rq5_3a[['SENSATIONALISM', "cnn2016_x",  "cnn2016_y", "cnn2020_x", "cnn2020_y", "fox2016_x", "fox2016_y", "fox2020_x", "fox2020_y", "All"]]
rq5_3a["All_p"]=rq5_3a["All"]/rq5_3a.All.sum()*100
rq5_3a["All_p"]=rq5_3a["All_p"].round(2)
rq5_3a.loc["All"] = rq5_3a.sum()
rq5_3a.to_csv('rq5_3a.csv')
rq5_3a

,SENSATIONALISM,cnn2016_x,cnn2016_y,cnn2020_x,cnn2020_y,fox2016_x,fox2016_y,fox2020_x,fox2020_y,All,All_p
0,no,51,79.69,117,98.32,17,89.47,58,76.32,243,87.41
1,yes,13,20.31,2,1.68,2,10.53,18,23.68,35,12.59
All,noyes,64,100.00,119,100.00,19,100.00,76,100.00,278,100.00


In [ ]:
print('SENSATIONALISM')
#CNNvFox2016
crosstab=pd.crosstab(dfstory2016['SENSATIONALISM'], dfstory2016['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2016: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNNvFox2020
crosstab=pd.crosstab(dfstory2020['SENSATIONALISM'], dfstory2020['PUBLISHER'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNNvFox2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# CNN2016v2020
crosstab=pd.crosstab(dfstoryCNN['SENSATIONALISM'], dfstoryCNN['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("CNN2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))
# FOX2016v2020
crosstab=pd.crosstab(dfstoryFOX['SENSATIONALISM'], dfstoryFOX['YEAR'])# crosstab with raw numbers
chi2, p, dof, expected = stats.chi2_contingency(crosstab)
print("FOX2016v2020: chi2 = "+ chi2.round(2).astype(str) + ", p = " + p.round(2).astype(str))

SENSATIONALISM
CNNvFox2016: chi2 = 0.4, p = 0.53
CNNvFox2020: chi2 = 22.06, p = 0.0
CNN2016v2020: chi2 = 16.8, p = 0.0
FOX2016v2020: chi2 = 0.89, p = 0.35


In [ ]:
#comparison of number of sources: Significant difference between CNN and FOX in 2020 (no data for 2016)
pg.ttest(dfstoryCNN2020.SRC_TOTAL, dfstoryFOX2020.SRC_TOTAL, paired=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.040818,166.75265,two-sided,0.042846,"[-0.65, -0.01]",0.295797,1.096,0.517768


In [ ]:
dfstoryCNN2020.describe().round(2)

,STORYNUMBER,CODER,YEAR,SRC_TOTAL
count,119.0,119.00,119.0,119.00
mean,125.7,1.87,2020.0,1.63
std,46.9,0.34,0.0,1.14
min,50.0,1.00,2020.0,0.00
25%,79.5,2.00,2020.0,1.00
50%,137.0,2.00,2020.0,2.00
75%,166.5,2.00,2020.0,2.00
max,198.0,2.00,2020.0,4.00


In [ ]:
dfstoryFOX2020.describe().round(2)

,STORYNUMBER,CODER,YEAR,SRC_TOTAL
count,76.00,76.00,76.0,76.00
mean,56.04,1.36,2020.0,1.96
std,43.91,0.48,0.0,1.08
min,1.00,1.00,2020.0,0.00
25%,19.75,1.00,2020.0,1.00
50%,38.50,1.00,2020.0,2.00
75%,106.25,2.00,2020.0,3.00
max,130.00,2.00,2020.0,5.00


In [ ]:
# #chisquaers
# pubyearA=['cnn2016_x','fox2016_x','cnn2016_x','cnn2020_x']
# pubyearB=['cnn2020_x','fox2020_x','fox2016_x','fox2020_x']
# tweetbyA=['trump_x','trump_x','trump_x','biden_x', 'biden_x','harris_x']
# tweetbyB=['biden_x','pence_x','harris_x','pence_x', 'harris_x','pence_x']
# pub=[rq1_1a, rq1_2a, rq2_1a, rq2_2a, rq3_1a, rq3_2a, rq3_3a, rq5_1a, rq5_2a, rq5_3a]
# candidate=[rq3_4a, rq3_5a, rq3_6a, rq3_7a]
# sig=" was significant (CHI2 = "
# nosig=" was not significant (CHI2 = "

# for h in pub:
#     for i, j in zip(pubyearA, pubyearB):
#         expected, observed, stats = pg.chi2_independence(h, x=i,y=j)
#         chi2=stats.iloc[0,2]
#         pval=stats.iloc[0,4]
#         if pval<0.05:
#             print("The relationship between " + i + " and " + j + " for " + h.columns[0] + sig + chi2.round(2).astype(str) + ", p = "+pval.round(2).astype(str)+").")
#         else:
#             print("The relationship between " + i + " and " + j+ " for " + h.columns[0] + nosig + chi2.round(2).astype(str) + ", p = "+pval.round(2).astype(str)+").")
# for k in candidate:
#     for i, j in zip(tweetbyA, tweetbyB):
#         expected, observed, stats = pg.chi2_independence(k, x=i,y=j)
#         chi2=stats.iloc[0,2]
#         pval=stats.iloc[0,4]
#         if pval<0.05:
#             print("The relationship between " + i + " and " + j + " for " + k.columns[0] + sig + chi2.round(2).astype(str) + ", p = "+pval.round(2).astype(str)+").")
#         else:
#             print("The relationship between " + i + " and " + j+ " for " + k.columns[0] + nosig + chi2.round(2).astype(str) + ", p = "+pval.round(2).astype(str)+").")

The relationship between cnn2016_x and cnn2020_x for cnn2016_x was not significant (CHI2 = 24.0, p = 0.24).
The relationship between fox2016_x and fox2020_x for cnn2016_x was not significant (CHI2 = 18.0, p = 0.12).
The relationship between cnn2016_x and fox2016_x for cnn2016_x was not significant (CHI2 = 18.0, p = 0.12).
The relationship between cnn2020_x and fox2020_x for cnn2016_x was not significant (CHI2 = 24.0, p = 0.24).
The relationship between cnn2016_x and cnn2020_x for STORYEMOTIONALFRAMES was not significant (CHI2 = 12.0, p = 0.21).
The relationship between fox2016_x and fox2020_x for STORYEMOTIONALFRAMES was not significant (CHI2 = 12.0, p = 0.21).
The relationship between cnn2016_x and fox2016_x for STORYEMOTIONALFRAMES was not significant (CHI2 = 12.0, p = 0.21).
The relationship between cnn2020_x and fox2020_x for STORYEMOTIONALFRAMES was not significant (CHI2 = 12.0, p = 0.21).
The relationship between cnn2016_x and cnn2020_x for TWEETGENERICFRAMESDOMINANT was not sign

AssertionError: columns are not in dataframe.